<a href="https://colab.research.google.com/github/benasphy/Deep-Learning/blob/main/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

# Generate dummy data
np.random.seed(42)
X_data = np.random.rand(100, 1)  # 100 samples, 1 feature
y_data = 3 * X_data + 2 + np.random.normal(0, 0.1, (100, 1))  # y = 3x + 2 + noise

# Define the model
class SimpleNN(tf.keras.Model):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(1, activation=None)  # One neuron, linear activation

    def call(self, inputs):
        return self.dense1(inputs)

# Initialize model and optimizer
model = SimpleNN()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Loss function: Mean Squared Error (MSE)
loss_fn = tf.keras.losses.MeanSquaredError()

# Training loop
epochs = 100
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        predictions = model(X_data)  # Forward pass
        loss = loss_fn(y_data, predictions)  # Calculate loss

    # Compute gradients
    gradients = tape.gradient(loss, model.trainable_variables)

    # Apply gradients to update weights
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.numpy():.4f}")

# Evaluate the model
print("Final weights:", model.dense1.weights)


Epoch 10, Loss: 0.0698
Epoch 20, Loss: 0.0192
Epoch 30, Loss: 0.0163
Epoch 40, Loss: 0.0142
Epoch 50, Loss: 0.0127
Epoch 60, Loss: 0.0116
Epoch 70, Loss: 0.0107
Epoch 80, Loss: 0.0100
Epoch 90, Loss: 0.0096
Epoch 100, Loss: 0.0092
Final weights: [<KerasVariable shape=(1, 1), dtype=float32, path=simple_nn_4/dense_4/kernel>, <KerasVariable shape=(1,), dtype=float32, path=simple_nn_4/dense_4/bias>]


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train / 255.0  # Normalize to [0, 1]
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)  # One-hot encode labels
y_test = to_categorical(y_test, num_classes=10)

# Define the RNN model
class RNNModel(tf.keras.Model):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn_layer = tf.keras.layers.SimpleRNN(128, activation='relu', return_sequences=False)
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.rnn_layer(inputs)
        return self.output_layer(x)

# Initialize model, loss function, and optimizer
model = RNNModel()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Compile model for convenience (for evaluation later)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Training loop
batch_size = 64
epochs = 10

# Dataset preparation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).shuffle(buffer_size=1024)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Training
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            predictions = model(x_batch)  # Forward pass
            loss = loss_fn(y_batch, predictions)  # Calculate loss

        gradients = tape.gradient(loss, model.trainable_variables)  # Backpropagation
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Update weights

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}")

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/3
Step 0, Loss: 2.3039
Step 100, Loss: 1.0310
Step 200, Loss: 0.8212
Step 300, Loss: 0.5551
Step 400, Loss: 0.2880
Step 500, Loss: 0.2601
Step 600, Loss: 0.3368
Step 700, Loss: 0.1361
Step 800, Loss: 0.1875
Step 900, Loss: 0.2607
Epoch 2/3
Step 0, Loss: 0.2363
Step 100, Loss: 0.1359
Step 200, Loss: 0.1167
Step 300, Loss: 0.2300
Step 400, Loss: 0.2118
Step 500, Loss: 0.1315
Step 600, Loss: 0.1994
Step 700, Loss: 0.1218
Step 800, Loss: 0.2886
Step 900, Loss: 0.1929
Epoch 3/3
Step 0, Loss: 0.0271
Step 100, Loss: 0.1546
Step 200, Loss: 0.1089
Step 300, Loss: 0.0991
Step 400, Loss: 0.0684
Step 500, Loss: 0.2782
Step 600, Loss: 0.0632
Step 700, Loss: 0.0746
Step 800, Loss: 0.1291
Step 900, Loss: 0.0675
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9576 - loss: 0.1458
Test Loss: 0.1221, Test Accuracy: 0.9654
